In [15]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import functools
import random
import time
import random
from pygame_visualization import VRPVisualizator

In [16]:
vizu = VRPVisualizator()

In [17]:
def manhattan(a, b):
    return sum(abs(val1-val2) for val1, val2 in zip(a,b))

In [18]:
random.seed(42)
def generate_coordinates(n_nodes: int, x_coords_range, y_coords_range, min_distance: float):
    generated_coords = [(0, 0)]
    vizu.add_node(0, 0)
    for i in range(n_nodes - 1):
        is_ok = False
        while not is_ok:
            coord = (random.uniform(*x_coords_range), random.uniform(*y_coords_range))
            is_ok = True
            for node in generated_coords:
                if manhattan(coord, node) < min_distance:
                    is_ok = False
        generated_coords.append(coord)
        vizu.add_node(coord[0], coord[1])
    return generated_coords

In [19]:
def generate_graph_matrix(length: int):
    coords = generate_coordinates(length, (-600, 600), (-400, 400), 2)
    matrix = np.empty((length, length))
    matrix[:] = np.nan
    for i in range(length):
        for j in range(length):
            if i != j:
                dist = manhattan(coords[i], coords[j])
                matrix[i][j] = dist
                matrix[j][i] = dist
    return matrix

In [20]:
vizu.clear_nodes()
graph = generate_graph_matrix(60)
random.seed()

In [21]:
vizu.set_scaling(1.3)

In [22]:
print(graph)

[[          nan  547.30355397  491.39622744 ...  257.78183124
   791.56826308  353.02584615]
 [ 547.30355397           nan  595.83696245 ...  289.52172274
  1004.24750076  900.32940013]
 [ 491.39622744  595.83696245           nan ...  306.31523971
  1282.96449052  360.68957307]
 ...
 [ 257.78183124  289.52172274  306.31523971 ...           nan
  1047.943423    610.80767739]
 [ 791.56826308 1004.24750076 1282.96449052 ... 1047.943423
            nan  922.27491745]
 [ 353.02584615  900.32940013  360.68957307 ...  610.80767739
   922.27491745           nan]]


In [23]:
#Evaporation factor for global update of pheromones
RHO = 0.2
#Evaporation factor for local update of pheromones
KAPPA = RHO
#Q
OMICRON = 1
#Impact of pheromones
ALPHA = 1
#Impact of weights
BETA = 2
#Initial pheromones
TAU = 1
#Exploration/exploitation trade off
EPSILON = 0.5

In [24]:
class Ant():
    def __init__(self, init_pos: int, max_capacity: int, vizualizator: VRPVisualizator) -> None:
        self.init_pos = init_pos
        self.vizualizator = vizualizator
        self.current_pos = init_pos
        self.current_path = [init_pos]
        self.paths_history = []
        self.full_nodes_history = [init_pos]
        self.can_continue = True
        self.max_capacity = max_capacity
        self.current_capacity = 0

    def add_current_to_paths_history(self) -> None:
        self.paths_history.append(self.current_path)
        self.current_path = [self.init_pos]
    
    def add_to_current_path(self, node) -> None:
        self.current_path.append(node)
        self.full_nodes_history.append(node)
        if self.vizualizator:
            self.vizualizator.set_path(self.full_nodes_history)

    def move(self, dest: list, pheromones: list) -> None:
        if self.current_capacity < self.max_capacity:
            choice = random.uniform(0, 1)

            if choice <= EPSILON:
                dest_picked = self.exploitation(dest, pheromones)
            else:
                dest_picked = self.biased_exploration(dest, pheromones)

            #No destination available
            if not dest_picked:
                #Only if we already have started a new path
                if self.current_capacity != 0:
                    self.add_to_current_path(self.init_pos)
                    self.add_current_to_paths_history()
                self.can_continue = False
                return

            self.update_pheromones_locally(pheromones, dest_picked)
            self.current_pos = dest_picked
            self.add_to_current_path(self.current_pos)
            self.current_capacity += 1
        else:
            self.current_pos = self.init_pos
            self.add_to_current_path(self.init_pos)
            self.add_current_to_paths_history()
            self.current_capacity = 0

    def exploitation(self, dest: list, pheromones: list) -> int:
        current_best_viability = None
        current_best_dest = None
        for i in range(len(dest)):
            if not np.isnan(dest[i]) and i not in self.full_nodes_history:
                viability = pheromones[i] * (OMICRON / dest[i])**BETA
                if not current_best_viability or viability > current_best_viability:
                    current_best_viability = viability
                    current_best_dest = i
        return current_best_dest

    def biased_exploration(self, dest: list, pheromones: list) -> int:
        probabilities = []
        denominator = 0
        
        #Calculate the denominator first
        for i in range(len(dest)):
            if not np.isnan(dest[i]) and i not in self.full_nodes_history:
                denominator += pheromones[i]**ALPHA * (OMICRON/dest[i])**BETA

        #Calculate probabilities of picking one path
        for node, length in enumerate(dest):
            if node in self.full_nodes_history:
                probabilities.append(0)
            elif not np.isnan(dest[node]):
                nominator = pheromones[node]**ALPHA * (OMICRON/dest[node])**BETA
                probabilities.append(nominator / denominator)
            else:
                probabilities.append(np.nan)
        
        #If there is no path available return false
        if np.nansum(probabilities) == 0:
            return None

        #Roulette wheel
        cumulative_sum = []
        for i in range(len(probabilities)):
            if np.isnan(probabilities[i]):
                cumulative_sum.append(np.nan)
            elif np.nansum(cumulative_sum) == 0:
                cumulative_sum.append(1.0)
            else:
                cumulative_sum.append(np.nansum(probabilities[i:len(probabilities)]))
        
        #Pick a destination
        rand = random.uniform(0, 1)
        dest_picked = None
        cumulative_sum.append(0)

        for i in range(0, len(cumulative_sum)-1):
            p = cumulative_sum[i]
            nextp = cumulative_sum[i+1] if not np.isnan(cumulative_sum[i+1]) else cumulative_sum[i+2] if not np.isnan(cumulative_sum[i+2]) else cumulative_sum[i+3]
        
            if not np.isnan(p) and rand <= p and rand >= nextp:
                dest_picked = i
        return dest_picked

    def update_pheromones_locally(self, pheromones: list, dest: int) -> None:
        #Apply the ACS local updating rule
        pheromones[dest] = (1-KAPPA) * pheromones[dest] + KAPPA * TAU


In [25]:
class ACO():
    def __init__(self, graph, k_trucks: int, start: int, vizualizator: VRPVisualizator = None) -> None:
        self.graph = graph
        self.start = start
        self.vizualizator = vizualizator
        self.current_best_tour = None
        self.current_best_length = float('inf')
        self.current_max_best_length = float('inf')
        self.pheromones = np.ones(graph.shape)
        self.max_capacity = len(graph[0]) // k_trucks
        if self.vizualizator:
            self.vizualizator.set_pheromones(self.pheromones)

    def run(self, iter: int) -> list:
        for i in range(iter):
            print('Tour:', i)
            self.tour_construction(15)
            self.global_update_pheromones()
        if self.vizualizator: 
            self.vizualizator.set_path(self.flatten_tour(self.current_best_tour))
        print('Best:', self.current_best_tour)
        print('Length:', self.current_best_length)
        return self.current_best_tour, self.current_best_length, self.current_max_best_length

    def tour_construction(self, ant_amount: int) -> None:
        ants = [Ant(self.start, self.max_capacity, self.vizualizator) for i in range(ant_amount)]
        while ants:
            for ant in ants:
                if ant.can_continue:
                    ant.move(self.graph[ant.current_pos], self.pheromones[ant.current_pos])
                else:
                    self.update_current_best(ant.paths_history)
                    ants.remove(ant)

    def update_current_best(self, paths):
        biggest_length = 0
        for path in paths:
            length = self.calc_path_distance(path)
            biggest_length = length if length > biggest_length else biggest_length

        tour_length = self.calc_tour_distance(paths)

        if self.current_best_tour is None:
            self.current_best_tour = paths
            self.current_best_length = tour_length
            self.current_max_best_length = biggest_length
        else:
            diff_percent = (tour_length - self.current_best_length) / self.current_best_length
            biggest_diff_percent = (biggest_length - self.current_max_best_length) / self.current_max_best_length
            coef = diff_percent*1 + biggest_diff_percent*0
            if coef < 0:
                print('NEW BEST:', paths)
                print('NEW MAX LENGTH:', biggest_length)
                print('NEW FULL LENGTH:', tour_length)
                self.current_best_tour = paths
                self.current_best_length = tour_length
                self.current_max_best_length = biggest_length

    def global_update_pheromones(self) -> None:
        #Evaporation
        self.pheromones *= 1 - RHO
        #New pheromones
        full_path = self.flatten_tour(self.current_best_tour)
        for i in range(len(full_path)-1):
            current_node = full_path[i]
            next_node = full_path[i+1]
            self.pheromones[current_node][next_node] += RHO * (OMICRON / self.current_best_length)
           #self.pheromones[next_node][current_node] += RHO * (OMICRON / self.current_best_length)
        if self.vizualizator:
            self.vizualizator.set_pheromones(self.pheromones)

    def calc_path_distance(self, full_path: list) -> float:
        distance = 0
        for i in range(len(full_path)-1):
            cur_path = full_path[i]
            next_path = full_path[i+1]
            if not np.isnan(self.graph[cur_path][next_path]):
                distance += self.graph[cur_path][next_path]
        return distance

    def calc_tour_distance(self, tour: list) -> float:
        return self.calc_path_distance(self.flatten_tour(tour))

    def flatten_tour(self, tour) -> list:
        return [n for path in tour for n in path]


In [26]:
all_results = []
for i in range(1):
    print('ITER:', i)
    start = time.time()
    aco = ACO(graph, 4, 0, vizu)
    best_tour, global_length, max_length = aco.run(5)
    vizu.set_path([n for path in best_tour for n in path])
    all_results.append((global_length, max_length))
    print('TOTAL TIME:', time.strftime('%H:%M:%S', time.gmtime(time.time()-start)))

ITER: 0
Tour: 0
NEW BEST: [[0, 32, 28, 8, 52, 57, 45, 14, 23, 24, 30, 54, 29, 46, 9, 6, 0], [0, 41, 25, 38, 7, 48, 34, 35, 43, 26, 12, 22, 40, 2, 5, 1, 0], [0, 18, 27, 36, 55, 17, 42, 59, 53, 51, 3, 11, 16, 33, 19, 20, 0], [0, 47, 39, 44, 49, 31, 21, 56, 13, 37, 4, 10, 15, 58, 50, 0]]
NEW MAX LENGTH: 4422.5405146439725
NEW FULL LENGTH: 15662.59079416198
NEW BEST: [[0, 29, 46, 54, 30, 48, 24, 23, 7, 43, 26, 12, 35, 18, 59, 53, 0], [0, 8, 28, 52, 38, 25, 41, 39, 3, 11, 19, 15, 20, 49, 44, 31, 0], [0, 32, 42, 17, 6, 36, 27, 55, 9, 22, 40, 2, 34, 21, 1, 47, 0], [0, 57, 33, 16, 58, 50, 13, 37, 56, 4, 10, 5, 45, 14, 51, 0]]
NEW MAX LENGTH: 4756.134344832073
NEW FULL LENGTH: 15446.007285884834
NEW BEST: [[0, 18, 54, 7, 23, 24, 45, 14, 34, 22, 40, 2, 35, 26, 43, 12, 0], [0, 27, 17, 55, 42, 36, 51, 9, 53, 59, 37, 56, 13, 39, 44, 49, 0], [0, 8, 52, 28, 3, 11, 16, 33, 19, 20, 15, 31, 50, 58, 47, 25, 0], [0, 32, 1, 21, 41, 6, 30, 48, 46, 29, 57, 5, 10, 4, 38, 0]]
NEW MAX LENGTH: 5081.541387335152


In [27]:
print(all_results)

[(12753.178826555715, 4255.767006078931)]
